In [1]:
ishp = '/mnt/d/autocls/pureobj.shp'

lulc_col = 'idclcplus'

folder = '/mnt/d/autocls/sat_s2_cmb18'

out_ref = '/mnt/d/autocls/thresholdstst/ref'

out_cls = '/mnt/d/autocls/thresholdstst'

iname = 'idx_i_'

thresholds = [(i + 1) / 10 for i in range(-10, 9)]

In [2]:
import os

from glass.pys.oss import lst_folders_subfiles, mkdir, fprop
from glass.rd.shp import shp_to_obj
from glass.wenv.grs import run_grass
from glass.pys.tm import now_as_str

In [3]:
# List images folder
imgs = lst_folders_subfiles(folder, filter_folder=iname, files_format='.tif')

ikeys = list(imgs.keys())
refrst = imgs[ikeys[0]][0]

In [4]:
refrst

'/mnt/d/autocls/sat_s2_cmb18/idx_i_20180130/evi_i_20180130.tif'

In [5]:
shpdf = shp_to_obj(ishp)

# Get classes
lulcs = list(shpdf[lulc_col].unique())

In [6]:
# Create GRASS GIS
loc = now_as_str()

gs = run_grass(folder, location=loc, srs=refrst)

import grass.script.setup as gsetup
gsetup.init(gs, folder, loc, 'PERMANENT')

'/tmp/tmp06i76801'

In [7]:
from glass.it.shp import shp_to_grs
from glass.it.rst import rst_to_grs, grs_to_mask, grs_to_rst
from glass.dtr.torst import grsshp_to_grsrst
from glass.rst.alg import grsrstcalc

In [8]:
# Send LULC Shapefiled to GRASS GIS
gshp = shp_to_grs(ishp, asCMD=True)

# LULC Shapefile to Raster
lulc_rst = grsshp_to_grsrst(
    gshp, lulc_col,
    f"rst_{gshp}"
)

# Get Mask
maskrst = grsrstcalc(f"({lulc_rst} * 0) + 1", 'lulcmask', ascmd=True)

# Create Mask
grs_to_mask(maskrst, overwrite=True)

```Python
trst = {
    'idx_i_20180130' : [
        {
            -0.9 : {'pos' : 'rastername', 'neg': 'rastername'},
            -0.8 : {'pos' : 'rastername', 'neg': 'rastername'},
            ...
        }
    ],
    'idx_i_201805005' : [
        {
            -0.9 : {'pos' : 'rastername', 'neg': 'rastername'},
            -0.8 : {'pos' : 'rastername', 'neg': 'rastername'},
            ...
        }
    ],
    ...
}
```

In [9]:
# Add Indexes rasters To GRASS
# And Create thresholds rasters

trst = {}
for k in imgs:
    trst[k] = []
    for i in range(len(imgs[k])):
        _out = mkdir(os.path.join(out_cls, k), overwrite=True)
        imgs[k][i] = rst_to_grs(imgs[k][i])

        td = {}

        for thl in thresholds:
            td[thl] = {}

            _thl = str(thl).replace('-', 'n').replace('.', '')
            
            td[thl]['pos'] = grsrstcalc(
                f"{imgs[k][i]} >= {str(thl)}, 1, 0",
                f"{imgs[k][i]}_pos_{str(_thl)}"
            )
            td[thl]['neg'] = grsrstcalc(
                f"{imgs[k][i]} < {str(thl)}, 1, 0",
                f"{imgs[k][i]}_neg_{str(_thl)}"
            )

            td[thl]['pos'] = grs_to_rst(
                td[thl]['pos'],
                os.path.join(_out, f"{td[thl]['pos']}.tif"),
                is_int=True
            )

            td[thl]['neg'] = grs_to_rst(
                td[thl]['neg'],
                os.path.join(_out, f"{td[thl]['neg']}.tif"),
                is_int=True
            )

        trst[k].append(td)

syntax error
Parse error
ERROR: parse error


CalledModuleError: Module run r.mapcalc r.mapcalc expression=evi_i_20180130_pos_n09 = evi_i_20180130 >= -0.9, 1, 0 region=current --o --q ended with error
Process ended with non-zero return code 1. See errors in the (error) output.

In [ ]:
# For each class, threshold and index
# Create two classification results
clsres = {}
for cls in lulcs:
    clsres[cls] = {}
    # Reclassify LULC Raster
    # Class = 1
    # No class = 0
    clsres[cls]['pos'] = grsrstcalc(f"{lulc_rst} == {str(cls)}, 1, 0", f'cls_{str(cls)}_pos')
    clsres[cls]['neg'] = grsrstcalc(f"{lulc_rst} == {str(cls)}, 0, 1", f'cls_{str(cls)}_neg')

    clsres[cls]['pos'] = grs_to_rst(
        clsres[cls]['pos'],
        os.path.join(out_ref, f"{td[thl]['pos']}.tif"),
        is_int=True
    )

    clsres[cls]['neg'] = grs_to_rst(
        clsres[cls]['neg'],
        os.path.join(out_ref, f"{td[thl]['neg']}.tif"),
        is_int=True
    )